<a href="https://colab.research.google.com/github/ssanchezgoe/iue_curso_ia/blob/main/nb_google_colab/S02_Busqueda_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <tr>
     <th><p><img alt="Colaboratory logo" height="120 px" src="http://www.redttu.edu.co/es/wp-content/uploads/2016/01/iue.png" align="left" hspace="10px" vspace="0px"></p></th> 
     <th><h1>  Resolución de problemas mediante búsqueda I </h1></th>
  </tr>


# Resolución de problemas mediante búsqueda

Este capítulo describe una clase de agente basado en objetivos llamado agente re- solvente-problemas. Los agentes resolventes-problemas deciden qué hacer para en- contrar secuencias de acciones que conduzcan a los estados deseables. Comenzamos de- finiendo con precisión los elementos que constituyen el «problema» y su «solución», y daremos diferentes ejemplos para ilustrar estas definiciones. Entonces, describimos diferentes algoritmos de propósito general que podamos utilizar para resolver estos pro- blemas y así comparar las ventajas de cada algoritmo. Los algoritmos son no informa- dos, en el sentido que no dan información sobre el problema salvo su definición. El Capítulo 4 se ocupa de los algoritmos de búsqueda informada, los que tengan cierta idea de dónde buscar las soluciones.

Este capítulo utiliza los conceptos de análisis de algoritmos. Los lectores no fami- liarizados con los conceptos de complejidad asintótica (es decir, notación O()) y la NP completitud, debería consultar el Apéndice A.


## Agentes resolventes-problemas


Los agentes inteligentes deben maximizar su medida de rendimiento, lo que puede simplificarse, algunas veces, si el agente puede elegir un objetivo y trata de satisfacerlo. 

Para un agente, los objetivos ayudan a organizar su comportamiento limitando las metas que intenta alcanzar. El primer paso para solucionar un problema es entonces **formular el objetivo**, basándose en la situación actual y la medida de rendimiento.

Consideremos un objetivo como un conjunto de estados del mundo (aquellos estados que satisfacen el objetivo). El agente debe encontrar qué secuencia de acciones permite obtener un estado objetivo. Dado un objetivo, la **formulación del problema** es el proceso de decidir qué acciones y estados tenemos que considerar. 

Supongamos que un carro de conducción autómata requiere llevar a un pasajero de Medellín a Cali. Existen dos alternativas iniciales, ir al sur y tomar la via a la pintada o la via a bolombolo. Si el agente inteligente no sábe si mas allá de la pintada o bolombolo hay algún tipo de conexión con la ciudad de Cali, deberá tomar una decisión al azar entre la vía de la pintada o la de bolombolo.

Supongamos que el agente cuenta con un mapa de Colombia (en su sistema operativo). El propósito del mapa es dotar al agente de información sobre los estados en los que podría encontrarse, así como las acciones a tomar. De esta forma, el agente puede usar esta información para considerar los siguientes estados de un viaje hipotético a través de cada una de las vias (Bolombolo o La Pintada).

En general, *un agente con distintas opciones inmediatas de valores desconocidos puede decidir qué hacer, examinando las diferentes secuencias posibles de acciones que le conduzcan a estados de valores conocidos, y entonces escoger la mejor secuencia*.

Al proceso de hallar una secuencia de acciones para logra un objetivo, o el máximo rendimiento, se le conoce cómo **búsqueda**. Un algoritmo de busqueda toma como entrada un problema y devuelve una **solución** de la forma *secuencia de acciones*. Una vez encontrada la secuencia solución se procede a la fase de **ejecución**.

Tenemos entonces tres parte claras en la consecución de un rendimiento máximo u objetivo:

1. Formular
2. Búsqueda
3. Ejecución

Estos pasos pueden ser 



<center><img src='https://github.com/ssanchezgoe/iue_curso_ia/blob/main/imagenes/S02_formular_buscar_ejecutar.png?raw=true'></center>


Algunos supuestos que hemos realizado en este pseudocódigo para el diseño de agente son:

1. El entorno es estático, ya que la formulación y búsqueda del problema se hace sin prestar atención a cualquier cambió que pueda pasarle al entorno.
2. El agente supone que se conoce el estado inicial; esto es cierto si el entorno es **observable**.


A lo largo de esta sección discutiremos:

1. Proceso de formulación del problema.
2. Algunos algoritmos para la función **Búsqueda**.
3. El ajente supone que el entorno es determinista. 

A lo largo de esta sección, estudiaremos los siguientes tópicos:

1. Formulación del problema.
2. Arlgoritmos para la función de búsqueda.




## Problemas y soluciones bien definidos

Un problema puede definirse por cuatro componentes:

1. El **estado inicial** en el que comienza el agente. Por ejemplo, el estado inicial para nuestro agente en Cali se describe como *En(Mellin)*.
2. Una descipción de las posibles **acciones** desponiples por el agente. La formulación más conocida utiliza una **función sucesor**. Si se tiene un estado particular $x$, la función $\text{SUCESOR-FN}(x)$ retorna un conjunto de pares $<acción, sucesor>$ donde cada acción es una de las acciones permitidas en el estado $x$ y cada sucesor es un estado que puede alcanzarse desde $x$, aplicando la acción. Supongamos el problema de ir de Arad a Bucarest en Rumanía, como se muestra en la siguiente figura. En este caso, la función sucesor (partiendo desde Arad) sería:

$${<Ir(Sibiu), En(Sibiu)>, <Ir(Timisoara), En(Timisoara)>, <Ir(Zerind), En(Zerind)>}$$

- **Espacio de estados**: Implícitamente el estado inicial y la función sucesor definen el espacio de estados del problema (el conjunto de todos los estados alcanzables desde el estado inicial). El espacio de estados forma un grafo en el cual los nodos son estados y las líneas entre los nodos son acciones. Un **camino** en el espacio de estados es una secuencia de estados conectados por una secuencia de acciones.

3. El **test objetivo**, el cual determina si un estado es un estado objetivo. Algunas ve- ces existe un conjunto explícito de posibles estados objetivo, y el test simplemente comprueba si el estado es uno de ellos. El objetivo del agente en Rumanía es el conjunto {En(Bucarest)}. Algunas veces el objetivo se especifica como una pro- piedad abstracta más que como un conjunto de estados enumerados explícitamente. Por ejemplo, en el ajedrez, el objetivo es alcanzar un estado llamado «jaque mate», donde el rey del oponente es atacado y no tiene escapatoria.

4. Una función costo del camino que asigna un costo numérico a cada camino. El agente resolvente de problemas elige una función costo que refleje nuestra medida de rendimiento. Para el agente que intenta llegar a Bucarest, el tiempo es esencial, así que el costo del camino puede describirse como su longitud en kilómetros. En este capítulo, suponemos que el costo del camino puede describirse como la suma de los costos de las acciones individuales a lo largo del camino. El costo indivi- dual de una acción a que va desde un estado x al estado y se denota por c(x,a,y). Los costos individuales para Rumanía se muestran en la Figura 3.2 como las distancias de las carreteras. Suponemos que los costos son no negativos .

Los elementos anteriores definen un problema y pueden unirse en una estructura de da- tos simple que se dará como entrada al algoritmo resolvente del problema. Una solución de un problema es un camino desde el estado inicial a un estado objetivo. La calidad de la solución se mide por la función costo del camino, y una solución óptima tiene el cos- to más pequeño del camino entre todas las soluciones.

<center><img src='https://github.com/ssanchezgoe/iue_curso_ia/blob/main/imagenes/S02_travel.png?raw=true'></center>





